# 05. MLOps: Prediction Logging

This notebook demonstrates how to log model predictions for future performance tracking.

## Setup

In [ ]:
import hopsworks
import pandas as pd
import joblib

# Connect to Hopsworks
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()

## Configuration

In [ ]:
# Configuration
feature_view_name = 'demand_features_view'
model_name = 'demand_forecaster'
version = 1

# Example item-location to demonstrate
item_id = 46374
location_id = 33182

## Initialize Prediction Logging

In [ ]:
# Get feature view
demand_fv = fs.get_feature_view(name=feature_view_name, version=version)

# Initialize batch scoring
demand_fv.init_batch_scoring(training_dataset_version=1)
print("✓ Initialized prediction logging")

## Log Predictions for One Model

In [ ]:
# Get the model
model_full_name = f"{model_name}_item{item_id}_loc{location_id}"
model = mr.get_best_model(name=model_full_name, metric="rmse", direction="min")

# Download and load model
model_dir = model.download()
model_obj = joblib.load(model_dir + "/model.pkl")

print(f"✓ Loaded model: {model_full_name}")

In [ ]:
# Get batch data
batch_data = demand_fv.get_batch_data()

# Filter for this item-location
mask = (batch_data['sp_id'] == item_id) & (batch_data['loc_id'] == location_id)
item_data = batch_data[mask].head(10)  # Just 10 rows for demo

if len(item_data) > 0:
    # Make predictions
    features = ['lagged_1day_demand_quantity', 'lagged_2day_demand_quantity']
    predictions = model_obj.predict(item_data[features])
    
    # Log predictions
    demand_fv.log(
        features=item_data,
        predictions=predictions,
        training_dataset_version=1,
        model=model
    )
    
    print(f"✓ Logged {len(predictions)} predictions")
else:
    print("No data found for this item-location")

## Materialize and Read Logs

In [ ]:
# Materialize the logs
demand_fv.materialize_log(wait=True)
print("✓ Materialized prediction logs")

# Read back the logs
logs = demand_fv.read_log()
print(f"\nLogged predictions shape: {logs.shape}")
logs.head()

## Summary

✅ Set up prediction logging for demand forecasting models

What we did:
- Initialized batch scoring on the feature view
- Made predictions with one model
- Logged predictions with features
- Materialized logs for future analysis

Next: Use these logs to calculate performance metrics when actuals arrive.